# Praca domowa nr 3
Adrianna Grudzień\
12.04.2021 r.

W ramach pracy domowej z przedmiotu Wstęp do uczenia maszynowego stworzyłam model klasyfikacyjny, który  potrafi przewidzieć, czy następnego dnia będzie padał deszcz, czy też nie.

Zbiór danych *australia.csv* zawiera codzienne obserwacje pogody z wielu australijskich stacji pogodowych.
Dane pochodzą z https://www.kaggle.com/jsphyg/weather-dataset-rattle-package. Natomiast nasz zbiór jest już przygotowany do pracy i nie zawiera brakujących wartości i kolumn z tekstem.

Podstawowa część zadania polega na wytrenowaniu dowolnych 3 klasyfikatorów i sprawdzeniu ich skuteczności. Raport zawiera:

* podział danych na zbiór treningowy i testowy
* nauczenie 3 dowolnych klasyfikatorów
* w każdym klasyfikatorze należy wybrać minimum jeden hiperparametr (nie trzeba go stroić - wystarczy się zapoznać z parametrami modelu)
* wykorzystanie przynajmniej 3 miar oceny jakości klasyfikatorów i wybór najlepszego z nich.

Skupiam się na klasyfikacji binarnej - czy jutro deszcz będzie padał, czy też nie.

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed = 42
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [61]:
data = pd.read_csv('australia.csv')
data.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
3,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
4,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0


Kolumny:

-    MinTemp - Minimalna temperatura [C]
-    MaxTemp - Maksymalna temperatura [C]
-    Rainfall - Suma opadów [mm]
-    Evaporation - Miara odparowywania [mm]
-    Sunshine - Suma czasu nasłonecznienia [h]
-    WindGustSpeed - Najwyższa prędkość wiatru [km/h]
-    WindSpeed9am - Prędkość wiatru o 9:00 [km/h]
-   WindSpeed3pm - Prędkość wiatru o 15:00 [km/h]
-    Humidity9am - Wilgotność o 9:00 [%]
-    Humidity3pm - Wilgotność o 15:00 [%]
-    Pressure9am - Ciśnienie atmosferyczne o 9:00 [hPa]
-    Pressure3pm - Ciśnienie atmosferyczne o 15:00 [hPa]
-   Cloud9am - Zachmurzenie o 9:00 [skala: 0 - słońce, 8 - całkowite zachmurzenie]
-    Cloud3pm - Zachmurzenie o 15:00 [skala: 0 - słońce, 8 - całkowite zachmurzenie]
-    Temp9am - Temperatura o 9:00 [C]
-    Temp3pm - Temperatura o 15:00 [C]
-    RainToday - Czy dzisiaj padał deszcz [0 - nie, 1 - tak]
-    Zmienna celu: RainTomorrow - Czy jutro będzie padał deszcz [0 - nie, 1 - tak]

In [62]:
data.info()
# rzeczywiście nie ma nuli i wszystkie zmienne są numeryczne

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56420 entries, 0 to 56419
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MinTemp        56420 non-null  float64
 1   MaxTemp        56420 non-null  float64
 2   Rainfall       56420 non-null  float64
 3   Evaporation    56420 non-null  float64
 4   Sunshine       56420 non-null  float64
 5   WindGustSpeed  56420 non-null  float64
 6   WindSpeed9am   56420 non-null  float64
 7   WindSpeed3pm   56420 non-null  float64
 8   Humidity9am    56420 non-null  float64
 9   Humidity3pm    56420 non-null  float64
 10  Pressure9am    56420 non-null  float64
 11  Pressure3pm    56420 non-null  float64
 12  Cloud9am       56420 non-null  float64
 13  Cloud3pm       56420 non-null  float64
 14  Temp9am        56420 non-null  float64
 15  Temp3pm        56420 non-null  float64
 16  RainToday      56420 non-null  int64  
 17  RainTomorrow   56420 non-null  int64  
dtypes: flo

### Modelowanie

In [63]:
y = np.array(data['RainTomorrow'])
X = data.drop(['RainTomorrow'],axis=1)
print(y[:100]) # większość wartości to 0
y.mean() # rzeczywiście - średnia jest zdeccydowanie przesunięta w stronę zera

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0]


0.22025877348457992

## Zbiór treningowy, walidacyjny i testowy¶
Prosty podział danych na część, na której uczymy model i na część która służy nam do sprawdzenia jego skuteczności.

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

In [65]:
print(X.shape,X_train.shape, X_val.shape, X_test.shape)

(56420, 17) (39494, 17) (11848, 17) (5078, 17)


# Klasyfikator 1.: `Dummy Classifier`

Parametr strategy może przyjmować wartości {“stratified”, “most_frequent”, “prior”, “uniform”, “constant”}.

Strategy to use to generate predictions.

   - “stratified”: generates predictions by respecting the training set’s class distribution.

   - “most_frequent”: always predicts the most frequent label in the training set.

   - “prior”: always predicts the class that maximizes the class prior (like “most_frequent”) and predict_proba returns the class prior.

   - “uniform”: generates predictions uniformly at random.

   - “constant”: always predicts a constant label that is provided by the user. This is useful for metrics that evaluate a non-majority class

In [66]:
from sklearn.dummy import DummyClassifier

dc = DummyClassifier(strategy='uniform', random_state=42)
dc.fit(X_train,y_train)
y_proba = dc.predict_proba(X_val)
y_hat = dc.predict(X_val)
print('y:     ' + str(y_hat[0:30]) + '\ny_hat: ' + str(y_val[0:30]))

y:     [0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1]
y_hat: [0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1]


In [67]:
# miary:
ac1 = accuracy_score(y_val,y_hat)
pr1 = precision_score(y_hat, y_val)
re1 = recall_score(y_hat, y_val)

print('accuracy: ',ac1, '\nprecision_score: ', pr1, '\nrecall_score: ', re1)

accuracy:  0.4977211343686698 
precision_score:  0.49430523917995445 
recall_score:  0.2198952879581152


# Klasyfikator 2.: `Decision Tree Classifier`

splitter{“best”, “random”}

The strategy used to choose the split at each node. Supported strategies are “best” to choose the best split and “random” to choose the best random split.

In [68]:
from sklearn.tree import DecisionTreeClassifier,plot_tree, export_graphviz
import graphviz

tree1 = DecisionTreeClassifier(splitter = 'best')
tree2 = DecisionTreeClassifier(splitter = 'random')

tree1.fit(X_train,y_train)
y_hat = tree1.predict(X_val)
tree2.fit(X_train,y_train)
y_hat2 = tree2.predict(X_val)

In [69]:
# miary:
ac2 = accuracy_score(y_val,y_hat)
pr2 = precision_score(y_val, y_hat)
re2 = recall_score(y_val, y_hat)
print("splitter = 'best'")
print('accuracy: ',ac2, '\nprecision_score: ', pr2, '\nrecall_score: ', re2)

ac2 = accuracy_score(y_val,y_hat2)
pr2 = precision_score(y_val, y_hat2)
re2 = recall_score(y_val, y_hat2)
print("splitter = 'random'")
print('accuracy: ',ac2, '\nprecision_score: ', pr2, '\nrecall_score: ', re2)

splitter = 'best'
accuracy:  0.7948176907494936 
precision_score:  0.5379723157500935 
recall_score:  0.5459377372817008
splitter = 'random'
accuracy:  0.7855334233625928 
precision_score:  0.5169152419061477 
recall_score:  0.5394836750189825


W tym przypadku lepsze wyniki dostajemy dla splitter = 'best'.

# Klasyfikator 3.: `Ada Boost Classifier`

n_estimatorsint, default=50

The maximum number of estimators at which boosting is terminated. In case of perfect fit, the learning procedure is stopped early.


In [70]:
from sklearn.ensemble import AdaBoostClassifier

ab = AdaBoostClassifier(n_estimators=100, random_state=0)
ab.fit(X_train, y_train)
y_hat = ab.predict(X_val)

ab2 = AdaBoostClassifier(n_estimators=50, random_state=0)
ab2.fit(X_train, y_train)
y_hat2 = ab.predict(X_val)

In [71]:
# miary:
print("n_estimators=100")
ac3 = accuracy_score(y_hat,y_val)
pr3 = precision_score(y_hat, y_val)
re3 = recall_score(y_hat, y_val)
print('accuracy: ',ac3, '\nprecision_score: ', pr3, '\nrecall_score: ', re3)

print("n_estimators=50")
ac3 = accuracy_score(y_hat2,y_val)
pr3 = precision_score(y_hat2, y_val)
re3 = recall_score(y_hat2, y_val)
print('accuracy: ',ac3, '\nprecision_score: ', pr3, '\nrecall_score: ', re3)

n_estimators=100
accuracy:  0.8539837947332883 
precision_score:  0.530751708428246 
recall_score:  0.7389006342494715
n_estimators=50
accuracy:  0.8539837947332883 
precision_score:  0.530751708428246 
recall_score:  0.7389006342494715


Wyniki są takie same, zatem uczenie musiało być zaprzestane przy liczbie estymatorów <= 50.

## Miary ocen jakości klasyfikatorów

`Accuracy` najlepsze (największe) dla klasyfikatora Ada Boost.
`Precision score` właściwie nigdzie nie jest duże, ale tym razem najlepsze dla Decision Tree.
`Recall score` najmniejsze dla Dummy, największe dla Ada Boost.

Trudno wybrać jeden najlepszy model i jak widać, przy okazji sprawdzania wyników dla innych wartości parametrów, każdy z modeli można w ten sposób ulepszyć. Na ten moment najlepsze wydają się modele `Decision Tree` oraz `Ada Boost`.